 < center >  In the name of God  < center > 

#  < center >  Sharif University of Technology  < center >  Department of Economics  < center > 
#  < center >  Quantitative Economics - Spring 2022  < center > Term Paper < center > 

##   < center >  Students:  < center > 
 < center >   Mohamad R. Manzaritavakoli  < center > 
 < center >   Shayan Ahmadi  < center > 

##   < center >  Professor:  < center > 
 < center >   Dr. Ebrahimnejad  < center > 

In [1]:
from scipy.stats import t
from glob import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import f as F
from  statsmodels.stats.stattools import durbin_watson as dw

In [2]:
rf = pd.DataFrame({"1372":[0.115], "1373":[0.115], 
                 "1374":[0.14], "1375":[0.14], 
                 "1376":[0.14], "1377":[0.14], 
                 "1378":[0.14], "1379":[0.14], 
                 "1380":[0.13], "1381":[0.13], 
                 "1382":[0.13], "1383":[0.13], 
                 "1384":[0.13], "1385":[0.16], 
                 "1386":[0.16], "1387":[0.1725], 
                 "1388":[0.145], "1389":[0.14], 
                 "1390":[0.20], "1391":[0.20], 
                 "1392":[0.20], "1393":[0.22], 
                 "1394":[0.27320783783783786], "1395":[0.24360708333333336], 
                 "1396":[0.17669560747663562], "1397":[ 0.24419008849557505], 
                 "1398":[0.21082175257731978],  "1399":[0.17285067708333335], 
                 "1400":[0.20740733333333336]})
Shares_num = pd.read_excel("E:\MBA\Semester 4\Quantitative Methodes in Finance\Project\stock.xlsx")
Shares_num.set_index("نماد",  inplace = True)
Div = pd.read_excel("E:\MBA\Semester 4\Quantitative Methodes in Finance\Project\Divs.xlsx")
Div.set_index("year",  inplace = True)
Shares_num

,تعداد سهام
نماد,
اپال,20000000000
اپرداز,1000000000
اتكام,3900000000
اتكاي,4000000000
اخابر,60000000000
...,...
وفتخار,120000000
ولتجار,1800000000
ولراز,61000000


In [3]:
Price = pd.DataFrame({})
date = pd.DataFrame({})
date["JALALI DATE"] = []
Price["JALALI DATE"] = []
path2 = r"E:\MBA\Semester 4\Quantitative Methodes in Finance\Project"
addresses = glob(path2 + "\*.csv")
for address in addresses:
    name = address.split("\\")[-1][:-4]
    data = pd.read_csv(address)
    data.rename(columns = {"CLOSE":name},  inplace = True)
    Price = Price.merge(data[["JALALI DATE",  name]],  on = "JALALI DATE",  how = "outer")
    date = date.merge(data[["JALALI DATE",  "DATE"]],  on = "JALALI DATE",  how = "outer")
Price.sort_values("JALALI DATE",  ignore_index = True,  inplace = True)
date.sort_values("JALALI DATE",  ignore_index = True,  inplace = True)
date = date.set_index("JALALI DATE").mean(axis = 1).reset_index()
date.rename(columns = {0:"DATE"},  inplace = True)
date["DATE"] = pd.to_datetime(date["DATE"],  format = "%Y%m%d")
date = date.resample("7D",  on = "DATE",  origin = "epoch").last()
date.drop(columns = ["DATE"],  inplace = True)
del addresses,  name,  data,  path2,  address
date

,JALALI DATE
DATE,
2001-03-22,13800108.0
2001-03-29,13800114.0
2001-04-05,13800122.0
2001-04-12,13800129.0
2001-04-19,13800205.0
...,...
2021-07-08,14000423.0
2021-07-15,14000428.0
2021-07-22,14000506.0


In [4]:
names = []
Returns = pd.DataFrame({})
market_cap_YDA = pd.DataFrame({})
market_cap_CLOSE = pd.DataFrame({})
ILLIQ = pd.DataFrame({})
end_year_market_cap = pd.DataFrame({})
Returns["JALALI DATE"] = []
market_cap_YDA["JALALI DATE"] = []
market_cap_CLOSE["JALALI DATE"] = []
ILLIQ["JALALI DATE"] = []
end_year_market_cap["year"] = []
path3 = r"E:\MBA\Semester 4\Quantitative Methodes in Finance\Project\Adjusted"
bourse = pd.read_excel("E:\MBA\Semester 4\Quantitative Methodes in Finance\Project\bourse.xlsx")
addresses = glob(path3 + "\*.csv")
for address in addresses:
    name = address.split("\\")[-1][0:-6]
    if name not in list(bourse["نماد"]):
        continue
    names.append(name)
    data = pd.read_csv(address)
    data["DATE"] = pd.to_datetime(data["DATE"],  format = "%Y%m%d")
    data.loc[:,  "YDA adj. CLOSE"] = [data.loc[0,  "YDA adj. CLOSE"] if (i  ==  0)
                                  else data.loc[i,  "YDA adj. CLOSE"] if abs(data.loc[i,  "YDA adj. CLOSE"]-data.loc[i-1,  "adj. CLOSE"]) > 1
                                  else data.loc[i-1,  "adj. CLOSE"] for i in list(data.index)]
    data.replace(0,  np.nan,  inplace = True)
#     data["Rcc"] = np.log(data["adj. CLOSE"]/data["YDA adj. CLOSE"])
    data["Return"] = data["adj. CLOSE"]/data["YDA adj. CLOSE"]-1
    data.replace(0,  np.nan,  inplace = True)  
    data = data[(data["Return"] > np.nanpercentile(data["Return"],  1,  interpolation = "higher"))&
              (data["Return"] < np.nanpercentile(data["Return"],  99,  interpolation = "lower"))]
#     data.drop(columns = "Return",  inplace = True)
    data["market_cap_YDA_adj"] = Shares_num.loc[name,  "تعداد سهام"]*data["YDA adj. CLOSE"]
    data["market_cap_CLOSE"] = Shares_num.loc[name,  "تعداد سهام"]*data["adj. CLOSE"]
####################################################################
#     data1=data[["DATE", "Rcc", "RIAL VOLUME"]].copy()
#     data1=data1.resample("7D", on = "DATE", origin = "epoch").sum()
#     data1["Return"] = np.exp(data1["Rcc"])-1
#     data1.drop(columns =  ["Rcc"],  inplace = True)
#     data2=data[["DATE", "market_cap_CLOSE"]].copy()
#     data2=data2.dropna().resample("7D", on = "DATE", origin = "epoch").last()
#     data2.drop(columns =  ["DATE"],  inplace = True)
#     data3=data[["DATE", "market_cap_YDA_adj"]].copy()
#     data3=data3.dropna().resample("7D", on = "DATE", origin = "epoch").first()
#     data3.drop(columns =  ["DATE"],  inplace = True)
#     data = pd.concat([data1, data2, data3, date], axis = 1, join = "inner")
#     data.reset_index(inplace = True)
#     data=data[data["JALALI DATE"] > 0].copy()
#     data.replace(0, np.nan,  inplace = True)
####################################################################
    data["ILLIQ"] = abs(data["Return"]/data["RIAL VOLUME"])*(10**10)
    Returns = Returns.merge(data[["JALALI DATE",  "Return"]],  on = "JALALI DATE",  how = "outer")
    Returns.rename(columns = {"Return":name},  inplace = True)
    market_cap_YDA = market_cap_YDA.merge(data[["JALALI DATE",  "market_cap_YDA_adj"]],  on = "JALALI DATE",  how = "outer")
    market_cap_YDA.rename(columns = {"market_cap_YDA_adj":name},  inplace = True)
    market_cap_CLOSE = market_cap_CLOSE.merge(data[["JALALI DATE",  "market_cap_CLOSE"]],  on = "JALALI DATE",  how = "outer")
    market_cap_CLOSE.rename(columns = {"market_cap_CLOSE":name},  inplace = True)
    ILLIQ = ILLIQ.merge(data[["JALALI DATE",  "ILLIQ"]],  on = "JALALI DATE",  how = "outer")
    ILLIQ.rename(columns = {"ILLIQ":name},  inplace = True)
    data["year"] = [int(str(i)[0:4]) for i in data["JALALI DATE"]]
    Min = data["year"].min()
    Max = data["year"].max()
    ey_mcap = pd.DataFrame({})
    for i in range(Min,  Max + 1):
        d = data[data["year"]  ==  i].copy()
        if(d["Return"][d["JALALI DATE"] > (i*10000 + 1200 + 14)].count() > 0 and d["RIAL VOLUME"][d["JALALI DATE"] > (i*10000 + 1200 + 14)].count() > 0):
            ey_mcap = ey_mcap.append({"year":i, name:d.dropna().iloc[-1, :]["market_cap_CLOSE"]},  ignore_index = True)
        else:
            ey_mcap = ey_mcap.append({"year":i, name:np.nan},  ignore_index = True)
    end_year_market_cap = end_year_market_cap.merge(ey_mcap,  on = "year",  how = "outer")
Returns.sort_values("JALALI DATE", ignore_index = True,  inplace = True)
market_cap_YDA.sort_values("JALALI DATE",  ignore_index = True,  inplace = True)
market_cap_CLOSE.sort_values("JALALI DATE",  ignore_index = True,  inplace = True)
ILLIQ.sort_values("JALALI DATE",  ignore_index = True,  inplace = True)
end_year_market_cap.sort_values("year",  ignore_index = True,  inplace = True)
end_year_market_cap.set_index("year",  inplace = True)
del path3,  addresses,  address,  name,  data,  Min,  Max,  d,  ey_mcap
# del data1, data2, data3
del bourse
DIVYLD = pd.DataFrame({})
for name in names:
    DIVYLD[name] = (Div[name]*10**6)/end_year_market_cap[name]*100
DIVYLD.set_index(end_year_market_cap.index, inplace = True)
del name

In [5]:
ILLIQ["year"] = [int(str(i)[0:4]) for i in ILLIQ["JALALI DATE"]]
AILLIQ = pd.DataFrame({})
MILLIQ = pd.DataFrame({})
L = []
for i in range(ILLIQ["year"].min(), ILLIQ["year"].max() + 1):
    sample = ILLIQ.drop(columns = "JALALI DATE")[ILLIQ["year"]  ==  i].copy()
    sample2 = pd.DataFrame(sample.drop(columns = "year").mean())
    sample2.rename(columns = {0:i},  inplace = True)
    L.append(sample2)
AILLIQ = pd.concat(L, axis = 1)
AILLIQ = AILLIQ.transpose(copy = True)
AILLIQ.index.name = "year"
del L,  sample,  sample2,  i
ILLIQ["ym"] = [int(str(i)[0:6]) for i in ILLIQ["JALALI DATE"]]
L = []
for i in range(ILLIQ["year"].min(), ILLIQ["year"].max() + 1):
    for j in range(1, 13):
        sample = ILLIQ.drop(columns = "JALALI DATE")[ILLIQ["ym"] == (i*100 + j)].copy()
        sample2 = pd.DataFrame(sample.drop(columns =  ["year", "ym"]).mean())
        sample2.rename(columns = {0:(i*100 + j)},  inplace = True)
        L.append(sample2)
MILLIQ = pd.concat(L, axis = 1)
MILLIQ = MILLIQ.transpose(copy = True)
MILLIQ.index.name = "ym"
del L, sample, sample2, i, j
Returns = Returns.copy()
SDReturns = pd.DataFrame()
Returns["year"] = [int(str(i)[0:4]) for i in Returns["JALALI DATE"]]
L = []
for i in range(Returns["year"].min(), Returns["year"].max() + 1):
    sample = Returns.drop(columns = "JALALI DATE")[Returns["year"]  ==  i].copy()
    sample2 = pd.DataFrame(sample.drop(columns = "year").std()*100)
    sample2.rename(columns = {0:i},  inplace = True)
    L.append(sample2)
SDReturns = pd.concat(L, axis = 1)
SDReturns = SDReturns.transpose(copy = True)
SDReturns.index.name = "year"
del L, sample, sample2, i, Returns
R100 = pd.DataFrame({})
R100Y = pd.DataFrame({})
M_Returns = pd.DataFrame({})
d_Rcc = np.log(Returns.set_index("JALALI DATE").copy() + 1).reset_index()
d_Rcc["year"] = [int(str(i)[0:4]) for i in d_Rcc["JALALI DATE"]]
L1= []
L2= []
for i in range(d_Rcc["year"].min(), d_Rcc["year"].max() + 1):
    sample=d_Rcc[d_Rcc["year"]  ==  i].copy()
    sample2 = pd.DataFrame(np.exp(sample[sample["JALALI DATE"] > (i*10000 + 918)].drop(columns =  ["year", "JALALI DATE"]).copy().sum(min_count = 1))-1)
    sample2.rename(columns = {0:i},  inplace = True)
    L1.append(sample2)
    sample3 = pd.DataFrame(np.exp(sample[sample["JALALI DATE"] < =(i*10000 + 918)].drop(columns =  ["year", "JALALI DATE"]).copy().sum(min_count = 1))-1)
    sample3.rename(columns = {0:i},  inplace = True)
    L2.append(sample3)
R100 = pd.concat(L1, axis = 1)
R100 = R100.transpose(copy = True)
R100.index.name = "year"
R100Y = pd.concat(L2, axis = 1)
R100Y = R100Y.transpose(copy = True)
R100Y.index.name = "year"
del L1, L2, sample, sample2, sample3, i
d_Rcc["ym"] = [int(str(i)[0:6]) for i in d_Rcc["JALALI DATE"]]
L= []
for i in range(d_Rcc["year"].min(), d_Rcc["year"].max() + 1):
    for j in range(1, 13):
        sample = d_Rcc[d_Rcc["ym"] == (i*100 + j)].copy()
        sample2 = pd.DataFrame(np.exp(sample.drop(columns =  ["year", "ym", "JALALI DATE"]).sum(min_count = 1))-1)
        sample2.rename(columns = {0:(i*100 + j)},  inplace = True)
        L.append(sample2)
M_Returns = pd.concat(L, axis = 1)
M_Returns = M_Returns.transpose(copy = True)
M_Returns.index.name = "ym"
del L, sample, sample2, i, j, d_Rcc

In [6]:
Returns = Returns.copy()
admitted_stocks =  []
show = pd.DataFrame({})
Returns["year"] = [int(str(i)[0:4]) for i in Returns["JALALI DATE"]]
Price["year"] = [int(str(i)[0:4]) for i in Price["JALALI DATE"]]
for i in range(Returns["year"].min(), Returns["year"].max() + 1):
    s_Returns = Returns[Returns["year"]  ==  i].copy()
    s_Price=Price[Price["year"]  ==  i].copy()
    s_ey_mcap = end_year_market_cap.loc[i, ].copy()
    s_AILLIQ = AILLIQ.loc[i, ].copy()
    stocks =  []
    stocks = names.copy()
    for stock in names:
        if((s_Price[stock].pct_change().replace(0, np.nan).count() < 200) 
           or (s_Price[stock].mean() < 50) or (np.isnan(s_ey_mcap[stock]))
          or s_AILLIQ[stock] <  = np.nanpercentile(s_AILLIQ, 1) or 
           s_AILLIQ[stock] >  = np.nanpercentile(s_AILLIQ, 99)):
            stocks.remove(stock)
    admitted_stocks.append(stocks)
show["year"] = [j for j in range(Returns["year"].min(), Returns["year"].max() + 1)]
show["admitted_Shares_num"] = [len(k) for k in admitted_stocks]
show["admitted_stocks"] = [i for i in admitted_stocks]
show.set_index("year",  inplace = True)
del Returns, admitted_stocks, i, s_Returns, s_Price, s_ey_mcap, s_AILLIQ, stock, stocks
show

,admitted_stocks_num,admitted_stocks
year,,
1380,1,[شكربن]
1381,2,"[كروي, ونيكي]"
1382,1,[ونيكي]
1383,24,"[بموتو, خاذين, خبهمن, خرينگ, خزاميا, خساپا, خو..."
1384,9,"[بكاب, تايرا, خزاميا, خودرو, فاذر, فاسمين, فسر..."
1385,3,"[بكاب, تايرا, وساخت]"
1386,8,"[خزاميا, خپارس, دجابر, شاراك, شبهرن, شكربن, فو..."
1387,6,"[بكاب, دالبر, دكوثر, شبهرن, وساخت, وپارس]"
1388,21,"[بترانس, بكاب, خاذين, ختوقا, دالبر, دسبحا, دكو..."


In [7]:
value_weighted_portfolios_Returns = pd.DataFrame({})
equally_weighted_portfolios_Returns = pd.DataFrame({})
BETA = pd.DataFrame(np.zeros((len(show.index), len(names))), index =  [i for i in range(show.index.min(), show.index.max() + 1)], 
                  columns = names)
BETA.replace(0, np.nan,  inplace = True)
BETA.index.name = "year"
market_AILLIQ = pd.DataFrame({})
market_MILLIQ = pd.DataFrame({})
Returns["year"] = [int(str(i)[0:4]) for i in Returns["JALALI DATE"]]
market_cap_CLOSE["year"] = [int(str(i)[0:4]) for i in market_cap_CLOSE["JALALI DATE"]]
market_cap_YDA["year"] = [int(str(i)[0:4]) for i in market_cap_YDA["JALALI DATE"]]
for i in range(show.index.min(), show.index.max() + 1):
    if ((i < 1389) or (i > 1399)):
        continue
    stocks = show.loc[i, "admitted_stocks"]
    market_AILLIQ.loc[i, "AILLIQ"]=AILLIQ.loc[i, stocks].mean()
    s_milliq = MILLIQ.loc[[int(str(z)[0:4])  ==  i for z in MILLIQ.index], stocks]
    for m in range(1, 13):
        mask2 = list((s_milliq.loc[(i*100 + m), ] >= np.nanpercentile(s_milliq.loc[(i*100 + m), ], 0)) & 
                   (s_milliq.loc[(i*100 + m), ] <= np.nanpercentile(s_milliq.loc[(i*100 + m), ], 99)))
        market_MILLIQ.loc[(i*100 + m), "MILLIQ"]=s_milliq.loc[(i*100 + m), mask2].mean()
    s_ey_mcap = end_year_market_cap.loc[i, stocks].copy()
    s_Returns = Returns.loc[list(Returns["year"]  ==  i), ["JALALI DATE"] + stocks].copy()
    s_mcap_CLOSE = Market_cap_CLOSE.loc[list(market_cap_CLOSE["year"]  ==  i), ["JALALI DATE"] + stocks].copy()
    s_mcap_YDA = Market_cap_YDA.loc[list(market_cap_YDA["year"]  ==  i), ["JALALI DATE"] + stocks].copy()
    vwp = pd.DataFrame({})
    ewp = pd.DataFrame({})
    vwp["JALALI DATE"]=s_mcap_CLOSE["JALALI DATE"].copy()
    ewp["JALALI DATE"]=s_Returns["JALALI DATE"].copy()
    vwp["market"]=s_mcap_CLOSE[stocks].sum(axis = 1)/s_mcap_YDA[stocks].sum(axis = 1)-1
    ewp["market"]=s_Returns[stocks].mean(axis = 1)
    market_corrcoeff=ewp["market"].corr(ewp["market"].shift(1))
    for j in range(1, 11):
        nst_decile= []
        mask=(s_ey_mcap > np.nanpercentile(s_ey_mcap, (j-1)*10))&(s_ey_mcap <  = np.nanpercentile(s_ey_mcap, j*10))
        nst_decile = list(s_ey_mcap.index[mask])
        if j == 1:
            nst_decile.append(list(s_ey_mcap.index[s_ey_mcap =  = np.nanpercentile(s_ey_mcap, 0)])[0])
        if j%2 == 0:
            vwp[str(j) + "st_decile"]=s_mcap_CLOSE[nst_decile].sum(axis = 1)/s_mcap_YDA[nst_decile].sum(axis = 1)-1
        ewp[str(j) + "st_decile"]=s_Returns[nst_decile].mean(axis = 1)
        B_sum=0
        for k in range(-1, 2):
            X=sm.add_constant(ewp["market"].copy())
            fit = sm.OLS(ewp[str(j) + "st_decile"], X.shift(k), missing = "drop").fit()
            B_sum  + = fit.params[1]
        BETA.loc[i, nst_decile]=B_sum/(1 + 2*market_corrcoeff)
    value_weighted_portfolios_Returns = value_weighted_portfolios_Returns.append(vwp, ignore_index = True)
    equally_weighted_portfolios_Returns = equally_weighted_portfolios_Returns.append(ewp, ignore_index = True)
value_weighted_portfolios_Returns.sort_values("JALALI DATE", ignore_index = True,  inplace = True)
value_weighted_portfolios_Returns.replace(0, np.nan,  inplace = True)
equally_weighted_portfolios_Returns.sort_values("JALALI DATE", ignore_index = True,  inplace = True)
market_AILLIQ.dropna(how = "all",  inplace = True)
BETA.dropna(how = "all",  inplace = True)
del i, s_ey_mcap, stocks, s_Returns, s_mcap_CLOSE, s_mcap_YDA, vwp, ewp, market_corrcoeff, j, nst_decile, mask, B_sum, X, fit, k, m, mask2, s_milliq
d_Rcc = np.log(value_weighted_portfolios_Returns.set_index("JALALI DATE").copy() + 1).reset_index()
d_Rcc["year"] = [int(str(i)[0:4]) for i in d_Rcc["JALALI DATE"]]
d_Rcc["ym"] = [int(str(i)[0:6]) for i in d_Rcc["JALALI DATE"]]
L= []
for i in range(d_Rcc["year"].min(), d_Rcc["year"].max() + 1):
    for j in range(1, 13):
        sample=d_Rcc[d_Rcc["ym"] == (i*100 + j)].copy()
        sample2 = pd.DataFrame(np.exp(sample.drop(columns =  ["year", "ym", "JALALI DATE"]).sum(min_count = 1))-1)
        sample2.rename(columns = {0:(i*100 + j)},  inplace = True)
        L.append(sample2)
M_Returns_vw_portfo = pd.concat(L, axis = 1)
M_Returns_vw_portfo = M_Returns_vw_portfo.transpose(copy = True)
M_Returns_vw_portfo.index.name = "ym"
M_Returns_vw_portfo["rf"] = [(1 + rf[str(i)[0:4]][0])**1/12-1 for i in M_Returns_vw_portfo.index]
del d_Rcc, L, i, j, sample, sample2
########################################
d_Rcc = np.log(equally_weighted_portfolios_Returns.set_index("JALALI DATE").copy() + 1).reset_index()
d_Rcc["year"] = [int(str(i)[0:4]) for i in d_Rcc["JALALI DATE"]]
d_Rcc["ym"] = [int(str(i)[0:6]) for i in d_Rcc["JALALI DATE"]]
L= []
for i in range(d_Rcc["year"].min(), d_Rcc["year"].max() + 1):
    for j in range(1, 13):
        sample=d_Rcc[d_Rcc["ym"] == (i*100 + j)].copy()
        sample2 = pd.DataFrame(np.exp(sample.drop(columns =  ["year", "ym", "JALALI DATE"]).sum(min_count = 1))-1)
        sample2.rename(columns = {0:(i*100 + j)},  inplace = True)
        L.append(sample2)
M_Returns_ew_portfo = pd.concat(L, axis = 1)
M_Returns_ew_portfo = M_Returns_ew_portfo.transpose(copy = True)
M_Returns_ew_portfo.index.name = "ym"
M_Returns_ew_portfo["rf"] = [(1 + rf[str(i)[0:4]][0])**1/12-1 for i in M_Returns_ew_portfo.index]
del d_Rcc, L, i, j, sample, sample2

In [8]:
cross_section = pd.DataFrame({})
for i in range(BETA.index.min() + 1, BETA.index.max() + 1):
    for name in show.loc[i-1, "admitted_stocks"]:
        c_s = pd.DataFrame({})
        for j in range(1, 13):
            c_s.loc[(i*100 + j), "name"]=name
            c_s.loc[(i*100 + j), "Return"] = M_Returns.loc[(i*100 + j), name]*100
            c_s.loc[(i*100 + j), "BETA"]=BETA.loc[i-1, name]
            c_s.loc[(i*100 + j), "ILLIQMA"]=AILLIQ.loc[i-1, name]/market_AILLIQ.loc[i-1, "AILLIQ"]
            c_s.loc[(i*100 + j), "R100"] = R100.loc[i-1, name]*100
            c_s.loc[(i*100 + j), "R100YR"] = R100Y.loc[i-1, name]*100
            c_s.loc[(i*100 + j), "LnSIZE"] = np.log(end_year_market_cap.loc[i-1, name])
            c_s.loc[(i*100 + j), "SDReturns"]=SDReturns.loc[i-1, name]
            c_s.loc[(i*100 + j), "DIVYLD"]=DIVYLD.loc[i-1, name]
        c_s.index.name = "ym"
        cross_section=cross_section.append(c_s.reset_index(), ignore_index = True)
cross_section.sort_values("ym", ignore_index = True,  inplace = True)
del i, j, c_s, name
cross_section

,ym,name,return,BETA,ILLIQMA,R100,R100YR,LnSIZE,SDRET,DIVYLD
0,139001,آسيا,-10.177743,0.842697,2.435646,6.769008,72.416980,28.387876,1.980167,3.735758
1,139001,خساپا,22.454222,1.139628,0.004918,38.749772,62.804976,31.021576,2.429760,17.518792
2,139001,ونوين,3.152091,1.139628,0.015802,-0.566409,47.733841,30.539045,1.073154,13.959733
3,139001,وغدير,27.205753,1.018653,0.005752,65.967977,36.141580,32.955182,1.870553,1.480004
4,139001,شپنا,0.824322,1.018653,0.009231,19.540651,3.179746,31.675298,1.467747,3.646632
...,...,...,...,...,...,...,...,...,...,...
18703,139912,كنور,4.657402,1.144919,0.187520,95.758087,62.650934,29.901068,2.530997,36.705813
18704,139912,كمنگنز,-7.866753,1.144919,0.397609,43.453390,265.309979,29.996266,3.075559,2.186229
18705,139912,خزاميا,-9.195402,1.091612,0.173129,67.342564,124.998000,30.513981,2.951515,0.000000
18706,139912,ختور,-0.585284,0.951575,0.916234,358.607023,435.312961,30.752862,3.374865,0.079337


In [9]:
regressors1= ["BETA", "ILLIQMA", "R100", "R100YR"]
regressors2= ["BETA", "ILLIQMA", "R100", "R100YR", "LnSIZE", "SDReturns", "DIVYLD"]
table1 = pd.DataFrame({})
table2 = pd.DataFrame({})
illiQ = pd.DataFrame({})
size = pd.DataFrame({})
divyld = pd.DataFrame({})
sdReturns = pd.DataFrame({})
regressors_corr = pd.DataFrame(np.zeros((4, 4)), columns =  ["ILLIQ", "LnSIZE", "DIVYLD", "SDReturns"], 
                                     index =  ["ILLIQ", "LnSIZE", "DIVYLD", "SDReturns"])
for i in range(BETA.index.min() + 1, BETA.index.max() + 1):
    stocks = show.loc[i-1, "admitted_stocks"].copy()
    dtf = pd.DataFrame({})
    dtf["ILLIQ"]=AILLIQ.loc[i-1, stocks]
    dtf["LnSIZE"] = np.log(end_year_market_cap.loc[i-1, stocks])
    dtf["DIVYLD"]=DIVYLD.loc[i-1, stocks]
    dtf["SDReturns"]=SDReturns.loc[i-1, stocks]
    regressors_corr=(regressors_corr + dtf.corr()).copy()
    illiq.loc[i-1, "mean"]=AILLIQ.loc[i-1, stocks].mean()
    illiq.loc[i-1, "s.d"]=AILLIQ.loc[i-1, stocks].std()
    illiq.loc[i-1, "skew"]=AILLIQ.loc[i-1, stocks].skew()
    size.loc[i-1, "mean"]=end_year_market_cap.loc[i-1, stocks].mean()/10**10
    size.loc[i-1, "s.d"]=end_year_market_cap.loc[i-1, stocks].std()/10**10
    size.loc[i-1, "skew"]=end_year_market_cap.loc[i-1, stocks].skew()/10**10
    divyld.loc[i-1, "mean"]=DIVYLD.loc[i-1, stocks].mean()
    divyld.loc[i-1, "s.d"]=DIVYLD.loc[i-1, stocks].std()
    divyld.loc[i-1, "skew"]=DIVYLD.loc[i-1, stocks].skew()    
    sdReturns.loc[i-1, "mean"]=SDReturns.loc[i-1, stocks].mean()
    sdReturns.loc[i-1, "s.d"]=SDReturns.loc[i-1, stocks].std()
    sdReturns.loc[i-1, "skew"]=SDReturns.loc[i-1, stocks].skew()
    for j in range(1, 13):
        c_s = cross_section[cross_section["ym"]  ==  i*100 + j]
        fit1=sm.OLS(c_s["Return"], sm.add_constant(c_s[regressors1].copy()), missing = "drop").fit()
        fit2=sm.OLS(c_s["Return"], sm.add_constant(c_s[regressors2].copy()), missing = "drop").fit()
        for exog1 in fit1.params.index:
            table1.loc[i*100 + j, exog1]=fit1.params[exog1]
        for exog2 in fit2.params.index:
            table2.loc[i*100 + j, exog2]=fit2.params[exog2]
result = pd.DataFrame({})
result.loc["ILLIQ", "Mean of annual means"]=illiq["mean"].mean()
result.loc["ILLIQ", "Mean of annual S.D"]=illiq["s.d"].mean()
result.loc["ILLIQ", "Median of annual means"]=illiq["mean"].median()
result.loc["ILLIQ", "Mean of annual skewness"]=illiq["skew"].mean()
result.loc["ILLIQ", "Min. annual mean"]=illiq["mean"].min()
result.loc["ILLIQ", "Max. annual mean"]=illiq["mean"].max()
##
result.loc["SIZE(BT)", "Mean of annual means"]=size["mean"].mean()
result.loc["SIZE(BT)", "Mean of annual S.D"]=size["s.d"].mean()
result.loc["SIZE(BT)", "Median of annual means"]=size["mean"].median()
result.loc["SIZE(BT)", "Mean of annual skewness"]=size["skew"].mean()
result.loc["SIZE(BT)", "Min. annual mean"]=size["mean"].min()
result.loc["SIZE(BT)", "Max. annual mean"]=size["mean"].max()
##
result.loc["DIVYLD(%)", "Mean of annual means"]=divyld["mean"].mean()
result.loc["DIVYLD(%)", "Mean of annual S.D"]=divyld["s.d"].mean()
result.loc["DIVYLD(%)", "Median of annual means"]=divyld["mean"].median()
result.loc["DIVYLD(%)", "Mean of annual skewness"]=divyld["skew"].mean()
result.loc["DIVYLD(%)", "Min. annual mean"]=divyld["mean"].min()
result.loc["DIVYLD(%)", "Max. annual mean"]=divyld["mean"].max()
##
result.loc["SDReturns(%)", "Mean of annual means"]=sdReturns["mean"].mean()
result.loc["SDReturns(%)", "Mean of annual S.D"]=sdReturns["s.d"].mean()
result.loc["SDReturns(%)", "Median of annual means"]=sdReturns["mean"].median()
result.loc["SDReturns(%)", "Mean of annual skewness"]=sdReturns["skew"].mean()
result.loc["SDReturns(%)", "Min. annual mean"]=sdReturns["mean"].min()
result.loc["SDReturns(%)", "Max. annual mean"]=sdReturns["mean"].max()
##
result.index.name = "Variable"
regressors_corr = Regressors_corr/(BETA.index.max()-BETA.index.min())
del regressors1, regressors2, illiq, size, divyld, sdReturns, stocks, exog1, exog2, fit1, fit2, dtf

In [10]:
regressors_corr

,ILLIQ,LnSIZE,DIVYLD,SDRET
ILLIQ,1.000000,-0.521321,-0.178526,0.245543
LnSIZE,-0.521321,1.000000,0.202451,-0.489621
DIVYLD,-0.178526,0.202451,1.000000,-0.433990
SDRET,0.245543,-0.489621,-0.433990,1.000000


In [11]:
result

,Mean of annual means,Mean of annual S.D,Median of annual means,Mean of annual skewness,Min. annual mean,Max. annual mean
Variable,,,,,,
ILLIQ,0.461109,1.554263e+00,0.214619,3.159614e+00,0.032541,2.679200
SIZE(BT),194187.381555,2.464991e+06,3487.762628,7.651925e-10,974.285296,794138.457370
DIVYLD(%),8.430709,7.152318e+00,8.273987,1.269424e+00,3.510148,11.722711
SDRET(%),2.162277,5.069906e-01,2.063495,-4.430592e-01,1.765145,2.911531


In [12]:
result2 = pd.DataFrame({})
s_p = BETA.index.min() + int((BETA.index.max()-BETA.index.min())/2)
exclude_farvardin= [int(str(i)[4:6])!=1 for i in table1.index]
f_h= [int(str(i)[0:4]) < =s_p for i in table1.index]
s_h= [int(str(i)[0:4]) > s_p for i in table1.index]
exclude_farvardin2= [int(str(i)[4:6])!=1 for i in table2.index]
f_h2= [int(str(i)[0:4]) < =s_p for i in table2.index]
s_h2= [int(str(i)[0:4]) > s_p for i in table2.index]
for v in table1.columns:
    result2.loc[v, "All months."] = Table1[v].mean()
    result2.loc[v + "_t_ratio", "All months."] = Table1[v].mean()/(table1[v].std()/np.sqrt(table1[v].count()))
    result2.loc[v, "Excl. farvardin."] = Table1.loc[exclude_farvardin, v].mean()
    result2.loc[v + "_t_ratio", "Excl. farvardin."] = Table1.loc[exclude_farvardin, v].mean()/(table1.loc[exclude_farvardin, v].std()/np.sqrt(table1.loc[exclude_farvardin, v].count()))
    result2.loc[v, str(BETA.index.min() + 1) + "_" + str(s_p) + "."] = Table1.loc[f_h, v].mean()
    result2.loc[v + "_t_ratio", str(BETA.index.min() + 1) + "_" + str(s_p) + "."] = Table1.loc[f_h, v].mean()/(table1.loc[f_h, v].std()/np.sqrt(table1.loc[f_h, v].count()))
    result2.loc[v, str(s_p) + "_" + str(BETA.index.max()) + "."] = Table1.loc[s_h, v].mean()
    result2.loc[v + "_t_ratio", str(s_p) + "_" + str(BETA.index.max()) + "."] = Table1.loc[s_h, v].mean()/(table1.loc[s_h, v].std()/np.sqrt(table1.loc[s_h, v].count()))
for v1 in table2.columns:
    result2.loc[v1, "All months"] = Table2[v1].mean()
    result2.loc[v1 + "_t_ratio", "All months"] = Table2[v1].mean()/(table2[v1].std()/np.sqrt(table2[v1].count()))
    result2.loc[v1, "Excl. farvardin"] = Table2.loc[exclude_farvardin2, v1].mean()
    result2.loc[v1 + "_t_ratio", "Excl. farvardin"] = Table2.loc[exclude_farvardin2, v1].mean()/(table2.loc[exclude_farvardin2, v1].std()/np.sqrt(table2.loc[exclude_farvardin2, v1].count()))
    result2.loc[v1, str(BETA.index.min() + 1) + "_" + str(s_p)] = Table2.loc[f_h2, v1].mean()
    result2.loc[v1 + "_t_ratio", str(BETA.index.min() + 1) + "_" + str(s_p)] = Table2.loc[f_h2, v1].mean()/(table2.loc[f_h2, v1].std()/np.sqrt(table2.loc[f_h2, v1].count()))
    result2.loc[v1, str(s_p) + "_" + str(BETA.index.max())] = Table2.loc[s_h2, v1].mean()
    result2.loc[v1 + "_t_ratio", str(s_p) + "_" + str(BETA.index.max())] = Table2.loc[s_h2, v1].mean()/(table2.loc[s_h2, v1].std()/np.sqrt(table2.loc[s_h2, v1].count()))
result2.index.name = "Variable"
del s_p, exclude_farvardin, f_h, s_h, exclude_farvardin2, f_h2, s_h2, v, v1

In [13]:
table1["ILLIQMA"].median()

0.3711279829234593

In [14]:
table1["ILLIQMA"].corr(table1["ILLIQMA"].shift(1))

0.027392824660115347

In [15]:
result2

,All months.,Excl. farvardin.,1390_1394.,1394_1399.,All months,Excl. farvardin,1390_1394,1394_1399
Variable,,,,,,,,
const,2.236707,2.343268,2.235623,2.237791,17.547696,20.006373,1.545903,33.549488
const_t_ratio,1.481247,1.458659,1.621320,0.828440,2.343848,2.527774,0.246256,2.514539
BETA,1.602989,1.091551,-0.544503,3.750481,-1.427759,-2.251821,0.173483,-3.029001
BETA_t_ratio,1.009026,0.658114,-0.408880,1.305667,-0.905640,-1.405037,0.128721,-1.062803
ILLIQMA,0.391285,0.377204,0.246193,0.536377,0.302971,0.287223,0.245397,0.360545
ILLIQMA_t_ratio,2.808506,2.571591,1.180249,2.905725,1.848102,1.654843,0.892148,1.995671
R100,-0.007697,-0.009332,0.004260,-0.019654,-0.006744,-0.006623,0.004761,-0.018249
R100_t_ratio,-1.104044,-1.280668,0.420351,-2.088987,-1.016732,-0.949876,0.472407,-2.159486
R100YR,-0.004830,-0.003658,-0.008835,-0.000825,-0.004420,-0.003515,-0.008161,-0.000679


In [16]:
result3 = pd.DataFrame({})
p_p = Table1["ILLIQMA"][table1["ILLIQMA"] > 0].count()/table1["ILLIQMA"].count()
p0=0.5
result3.loc["ILLIQMA", "median"] = Table1["ILLIQMA"].median()
result3.loc["ILLIQMA", "positive coeffs(%)"]=p_p*100
result3.loc["ILLIQMA", "Z value"]=(p_p-p0)/np.sqrt(p0*(1-p0)/table1["ILLIQMA"].count())
result3

,median,positive coeffs(%),Z value
ILLIQMA,0.371128,60.0,2.19089


In [23]:
a = pd.DataFrame({})
b = pd.DataFrame({})
c = pd.DataFrame({})
result7 = pd.DataFrame({})
def chowtest(testdata, splitpoint, k):
    td = np.split(testdata, [splitpoint])
    td2 = np.split(testdata.shift(1), [splitpoint])
    FIt = sm.OLS(testdata["lnMILLIQ"], sm.add_constant(testdata["lnMILLIQ"].shift(1).copy()), missing = "drop").fit()
    FIT1=sm.OLS(td[0]["lnMILLIQ"], sm.add_constant(td2[0]["lnMILLIQ"].copy()), missing = "drop").fit()
    FIT2=sm.OLS(td[1]["lnMILLIQ"], sm.add_constant(td2[1]["lnMILLIQ"].copy()), missing = "drop").fit()
    RSs = FIT.ssr
    RSS1=FIT1.ssr
    RSS2=FIT2.ssr
    t = len(list(testdata.index))-1
    t_statistic=((RSS-(RSS1 + RSS2))/(RSS1 + RSS2))*((T-2*k)/k)
    Return F.sf(t_statistic, k, (T-2*k))
#####################################################################################
lnMILLIQ = np.log(market_MILLIQ)
lnMILLIQ.rename(columns = {"MILLIQ":"lnMILLIQ"},  inplace = True)
lnMILLIQ = lnMILLIQ[(lnMILLIQ.index < =139712)]
time_series = pd.DataFrame({})
time_series["RM-Rf"]=(M_Returns_ew_portfo["market"]-M_Returns_ew_portfo["rf"])*100
j=1
for column in M_Returns_ew_portfo.columns:
    if column= = "market" or column= = "rf" or int(column.split("s")[0])%2 == 0:
        continue
    time_series["RSZ" + str(j) + "-Rf"]=(M_Returns_ew_portfo[column]-M_Returns_ew_portfo["rf"])*100
    j + =2
time_series = time_series[time_series.index < =139712]
time_series["lnMILLIQm-1"] = lnMILLIQ.shift(1).copy()
del j, column
######################################################################################
pvalue=0.05
st_break_ym=0
for i in lnMILLIQ.index[24:-23]:
    p_v=chowtest(lnMILLIQ, lnMILLIQ.index.get_loc(i), 2).copy()
    a.loc[i, "p_value"]=p_v
    if(p_v < =pvalue):
        st_break_ym=i
        pvalue=p_v
# you can see "a",  it shows that the p value in several points is below 0.01 level. I set the latest time.
st_break_ym=139410
del i, p_v
lnMILLIQ_lag = lnMILLIQ.shift(1).copy()
dt1 = np.split(lnMILLIQ, [lnMILLIQ.index.get_loc(st_break_ym)])
dt2 = np.split(lnMILLIQ_lag, [lnMILLIQ.index.get_loc(st_break_ym)])
# ######################################################################################
# # with this box I found that the years 1398 and 1399 have the unusual and extraordinary impact on the coeffs and the main regression too.
# pvalue1=0.05
# st_break_ym1=0
# for i in dt1[1].index[24:-23]:
#     p_v1=chowtest(dt1[1], dt1[1].index.get_loc(i), 2).copy()
#     b.loc[i, "p_value"]=p_v1
#     if(p_v1 < =pvalue1):
#         st_break_ym1=i
#         pvalue1=p_v1
# #st_break_ym1=139801
# del i, p_v1
# dt3 = np.split(dt1[1], [dt1[1].index.get_loc(st_break_ym1)])
# dt4 = np.split(dt2[1], [dt1[1].index.get_loc(st_break_ym1)])
# fit = sm.OLS(dt3[0], sm.add_constant(dt4[0].copy()), missing = "drop").fit()
# slope_adjusted=fit.params[1] + (1 + 3*fit.params[1])/(len(list(dt3[0].index)))
# const_adjusted=dt3[0].mean()[0]-slope_adjusted*dt4[0].mean()[0]
# lnMILLIQ_unexpected2=dt3[0]-const_adjusted-slope_adjusted*dt4[0]
# fit1=sm.OLS(dt3[1], sm.add_constant(dt4[1].copy()), missing = "drop").fit()
# slope_adjusted1=fit1.params[1] + (1 + 3*fit1.params[1])/(len(list(dt3[1].index)))
# const_adjusted1=dt3[1].mean()[0]-slope_adjusted1*dt4[1].mean()[0]
# lnMILLIQ_unexpected3=dt3[1]-const_adjusted1-slope_adjusted1*dt4[1]
# #########################################################################################
fit2=sm.OLS(dt1[1], sm.add_constant(dt2[1].copy()), missing = "drop").fit()
slope_adjusted2=fit2.params[1] + (1 + 3*fit2.params[1])/(len(list(dt1[1].index)))
const_adjusted2=dt1[1].mean()[0]-slope_adjusted2*dt2[1].mean()[0]
lnMILLIQ_unexpected2=dt1[1]-const_adjusted2-slope_adjusted2*dt2[1]
result7.loc["139410-139712", "const"]=fit2.params[0]
result7.loc["139410-139712", "const_t_ratio"]=fit2.tvalues[0]
result7.loc["139410-139712", "slope"]=fit2.params[1]
result7.loc["139410-139712", "slope_tratio"]=fit2.tvalues[1]
result7.loc["139410-139712", "R2"]=fit2.rsquared
result7.loc["139410-139712", "D-W"]=dw(fit2.resid)
###########################################################################################
fit3=sm.OLS(dt1[0], sm.add_constant(dt2[0].copy()), missing = "drop").fit()
slope_adjusted3=fit3.params[1] + (1 + 3*fit3.params[1])/(len(list(dt1[0].index))-1)
const_adjusted3=dt1[0].mean()[0]-slope_adjusted3*dt2[0].mean()[0]
lnMILLIQ_unexpected=dt1[0]-const_adjusted3-slope_adjusted3*dt2[0]
result7.loc["138901-139410", "const"]=fit3.params[0]
result7.loc["138901-139410", "const_t_ratio"]=fit3.tvalues[0]
result7.loc["138901-139410", "slope"]=fit3.params[1]
result7.loc["138901-139410", "slope_tratio"]=fit3.tvalues[1]
result7.loc["138901-139410", "R2"]=fit3.rsquared
result7.loc["138901-139410", "D-W"]=dw(fit3.resid)
#########################################################################################
time_series["lnMILLIQmu"] = pd.concat([lnMILLIQ_unexpected, lnMILLIQ_unexpected2
#                                      , lnMILLIQ_unexpected3
                                    ], axis = 0)
time_series["FARDUM"] = [1 if int(str(i)[4:]) == 1 else 0 for i in time_series.index]
del dt1, dt2
del fit3
del fit2
# del dt3, dt4, fit, fit1
regressors =  ["lnMILLIQm-1", "lnMILLIQmu", "FARDUM"]
############################################################################################################
## this test show that the coeffs have dramatically changed during 1398 and 1399. I droped them from the time_series_regression
## if you are curious to see what happen, put back this section and reletive lines in code,  also remove the filter above.
# def chowtest(testdata, splitpoint, k):
#     td = np.split(testdata, [splitpoint])
#     td2 = np.split(testdata.shift(1), [splitpoint])
#     FIt = sm.OLS(testdata["RM-Rf"], sm.add_constant(testdata[regressors].shift(1).copy()), missing = "drop").fit()
#     FIT1=sm.OLS(td[0]["RM-Rf"], sm.add_constant(td2[0][regressors].copy()), missing = "drop").fit()
#     FIT2=sm.OLS(td[1]["RM-Rf"], sm.add_constant(td2[1][regressors].copy()), missing = "drop").fit()
#     RSs = FIT.ssr
#     RSS1=FIT1.ssr
#     RSS2=FIT2.ssr
#     t = len(list(testdata.index))-1
#     t_statistic=((RSS-(RSS1 + RSS2))/(RSS1 + RSS2))*((T-2*k)/k)
#     Return F.sf(t_statistic, k, (T-2*k))
# pvalue2=0.05
# st_break_ym2=0
# for i in time_series.index[24:-23]:
#     p_v2=chowtest(time_series, time_series.index.get_loc(i), len(regressors) + 1).copy()
#     c.loc[i, "p_value"]=p_v2
#     if(p_v2 < =pvalue2):
#         st_break_ym2=i
#         pvalue2=p_v2
# #st_break_ym2=139801
# del i, p_v2
#############################################################################################################
result4 = pd.DataFrame({})
rs = pd.DataFrame({})
ts = np.split(time_series, 4)
# ts = np.split(time_series, [time_series.index.get_loc(st_break_ym)
#                          , time_series.index.get_loc(st_break_ym1)
#                         ])
ts.append(time_series.copy())
fit2=sm.OLS(ts[0]["RM-Rf"], sm.add_constant(ts[0][regressors]), missing = "drop").fit(cov_type = "HC3")
fit3=sm.OLS(ts[1]["RM-Rf"], sm.add_constant(ts[1][regressors]), missing = "drop").fit(cov_type = "HC3")
fit4=sm.OLS(ts[2]["RM-Rf"], sm.add_constant(ts[2][regressors]), missing = "drop").fit(cov_type = "HC3")
fit6=sm.OLS(ts[3]["RM-Rf"], sm.add_constant(ts[3][regressors]), missing = "drop").fit(cov_type = "HC3")
rs["lnMILLIQm-1"] = [fit2.params["lnMILLIQm-1"], fit3.params["lnMILLIQm-1"]
                           , fit4.params["lnMILLIQm-1"]
                           , fit6.params["lnMILLIQm-1"]
                          ]
rs["lnMILLIQmu"] = [fit2.params["lnMILLIQmu"], fit3.params["lnMILLIQmu"]
                          , fit4.params["lnMILLIQmu"]
                          , fit6.params["lnMILLIQmu"]
                         ]
result4["Mean"] = Rs.mean()
result4.rename(columns = {0:"Mean"},  inplace = True)
result4["Median"] = Rs.median()
result4.rename(columns = {0:"Median"},  inplace = True)
result4.index.name = "RM-Rf"
result5= [pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({})]
for i in range(5):
    for endog in ts[i].columns:
        if endog in regressors:
            continue
        fit = sm.OLS(ts[i][endog], sm.add_constant(ts[i][regressors]), missing = "drop").fit(cov_type = "HC3")
        fit5=sm.OLS(ts[i][endog], sm.add_constant(ts[i][regressors]), missing = "drop").fit()
        for reg in fit.params.index:
            result5[i].loc[reg, endog]=fit.params[reg]
            result5[i].loc[reg + "_t_ratio", endog]=fit5.tvalues[reg]
            result5[i].loc[reg + "_t_ratio_White S.D", endog]=fit.tvalues[reg]
        result5[i].loc["R2", endog]=fit.rsquared
        result5[i].loc["D_W", endog]=dw(fit.resid)
del i, endog, fit, fit2, fit3, fit5, reg, regressors
del fit4
main_result5 = Result5[-1].copy()
main_result5.index.name = "1389_1397"

In [18]:
result7

,const,const_t_ratio,slope,slope_tratio,R2,D-W
139410-139712,-0.644856,-2.385047,0.699652,5.318887,0.433302,2.059935
138901-139410,-0.378765,-3.172208,0.727016,8.614781,0.529292,1.839624


In [19]:
result6 = pd.DataFrame({})
result6.loc["1389/01_1394/10", "intercept"]=const_adjusted3
result6.loc["1389/01_1394/10", "slope_lnMILLIQm-1"]=slope_adjusted3
result6.loc["1394/10_1397/12", "intercept"]=const_adjusted2
result6.loc["1394/10_1397/12", "slope_lnMILLIQm-1"]=slope_adjusted2
result6.index.name = "lnMILLIQm"
result6

,intercept,slope_lnMILLIQm-1
lnMILLIQm,,
1389/01_1394/10,-0.311654,0.773796
1394/10_1397/12,-0.482699,0.779112


In [24]:
rs

,lnMILLIQm-1,lnMILLIQmu
0,-6.437516,-13.408937
1,6.386086,-16.998214
2,-2.345845,-22.457632
3,8.123209,-21.963870


In [20]:
result4

,Mean,Median
RM-Rf,,
lnMILLIQm-1,1.431483,2.020120
lnMILLIQmu,-18.707163,-19.481042


In [31]:
main_result5

,RM-Rf,RSZ1-Rf,RSZ3-Rf,RSZ5-Rf,RSZ7-Rf,RSZ9-Rf
1389_1397,,,,,,
const,95.680413,96.686659,95.309522,95.285615,94.269103,94.791650
const_t_ratio,39.974461,31.646027,32.265012,34.120049,34.543489,38.099124
const_t_ratio_White S.D,42.702350,32.890628,32.554526,35.308929,33.745491,41.650270
lnMILLIQm-1,2.281566,3.396851,2.371920,1.970179,1.362189,1.435238
lnMILLIQm-1_t_ratio,1.604234,1.871131,1.351359,1.187307,0.840058,0.970832
lnMILLIQm-1_t_ratio_White S.D,1.826614,2.087523,1.457909,1.323107,0.873194,1.099909
lnMILLIQmu,-17.357832,-20.097479,-20.662176,-17.218005,-16.867379,-14.264608
lnMILLIQmu_t_ratio,-6.005524,-5.447410,-5.792519,-5.105767,-5.118472,-4.747889
lnMILLIQmu_t_ratio_White S.D,-6.357080,-5.548384,-5.888519,-5.363794,-5.235868,-5.148708


,lnMILLIQ
138901,-1.148556
138902,-0.933153
138903,-1.148727
138904,-1.330046
138905,-1.322638
...,...
139708,-2.216751
139709,-2.208244
139710,-2.608187
139711,-2.534555


,RM-Rf,RSZ1-Rf,RSZ3-Rf,RSZ5-Rf,RSZ7-Rf,RSZ9-Rf
const,85.628561,81.856984,85.357877,84.400136,83.379760,89.596767
const_t_ratio,12.409219,9.243034,10.302616,9.987726,11.800548,11.369772
const_t_ratio_White S.D,13.346842,8.510075,10.248851,9.588872,13.115288,15.590680
lnMILLIQm-1,-2.345845,-3.882689,-2.232249,-3.047477,-3.946694,-0.516543
lnMILLIQm-1_t_ratio,-0.630099,-0.812596,-0.499379,-0.668417,-1.035282,-0.121493
lnMILLIQm-1_t_ratio_White S.D,-0.744543,-0.768761,-0.499917,-0.687973,-1.288047,-0.171890
lnMILLIQmu,-22.457632,-21.425442,-27.342231,-23.231577,-24.425868,-16.770735
lnMILLIQmu_t_ratio,-4.137071,-3.075331,-4.195088,-3.494663,-4.394351,-2.705296
lnMILLIQmu_t_ratio_White S.D,-5.049531,-3.004187,-4.139797,-3.822361,-5.972501,-2.665530
FARDUM,4.537620,0.432498,6.392719,3.706457,4.157086,2.991723
